In [1]:
%matplotlib inline

import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

from IPython.display import display

startup_df = pd.read_csv("Startup_Data.csv")
display(startup_df.head())


list(startup_df)

startup_df.groupby( ["Dependent-Company Status"] ).count()

,Company_Name,Dependent-Company Status,year of founding,Age of company in years,Internet Activity Score,Short Description of company profile,Industry of company,Focus functions of company,Investors,Employee Count,...,Percent_skill_Data Science,Percent_skill_Business Strategy,Percent_skill_Product Management,Percent_skill_Sales,Percent_skill_Domain,Percent_skill_Law,Percent_skill_Consulting,Percent_skill_Finance,Percent_skill_Investment,Renown score
0,Company1,Success,No Info,No Info,-1.0,Video distribution,NaN,operation,KPCB Holdings|Draper Fisher Jurvetson (DFJ)|Kl...,3.0,...,0,0,0,0,0,0,0,0,0,0
1,Company2,Success,2011,3,125.0,NaN,Market Research|Marketing|Crowdfunding,"Marketing, sales",NaN,NaN,...,8.823529412,21.76470588,10.88235294,2.941176471,0,0,0,0,0,8
2,Company3,Success,2011,3,455.0,Event Data Analytics API,Analytics|Cloud Computing|Software Development,operations,TechStars|Streamlined Ventures|Amplify Partner...,14.0,...,3.846153846,17.09401709,9.401709402,0,2.777777778,0,0,0,0,9
3,Company4,Success,2009,5,-99.0,The most advanced analytics for mobile,Mobile|Analytics,Marketing & Sales,Michael Birch|Max Levchin|Sequoia Capital|Keit...,45.0,...,0,0,0,0,0,0,0,0,0,5
4,Company5,Success,2010,4,496.0,The Location-Based Marketing Platform,Analytics|Marketing|Enterprise Software,Marketing & Sales,DFJ Frontier|Draper Nexus Ventures|Gil Elbaz|A...,39.0,...,0,0,0,0,0,0,0,0,0,6


,Company_Name,year of founding,Age of company in years,Internet Activity Score,Short Description of company profile,Industry of company,Focus functions of company,Investors,Employee Count,Employees count MoM change,...,Percent_skill_Data Science,Percent_skill_Business Strategy,Percent_skill_Product Management,Percent_skill_Sales,Percent_skill_Domain,Percent_skill_Law,Percent_skill_Consulting,Percent_skill_Finance,Percent_skill_Investment,Renown score
Dependent-Company Status,,,,,,,,,,,,,,,,,,,,,
Failed,167,167,125,123,93,98,139,95,70,58,...,167,167,167,167,167,167,167,167,167,167
Success,305,305,303,284,230,250,303,237,236,209,...,305,305,305,305,305,305,305,305,305,305


In [193]:
success_df = startup_df[startup_df["Dependent-Company Status"]=="Success"].reset_index()
failed_df = startup_df[startup_df["Dependent-Company Status"]=="Failed"].reset_index()

def normalize_double_hist(data, sizes, bins = 20, h_range = None):
    if (h_range == None):
        min_val = min(data[0].min(),data[1].min())
        max_val = max(data[0].max(),data[1].max())
    else:
        min_val = h_range[0]
        max_val = h_range[1]
    
    # if there are both + and - values center on 0, 0 has often a special meaning 
    # so it sholdn't be in a centre of a bin    
    if (min_val * max_val) < 0:
        max_val = max(abs(min_val),max_val)
        min_val = -max_val
        
    #get histograms
    hist0, bin_edges = np.histogram(data[0], bins, range =  (min_val,max_val))
    hist1, _ = np.histogram(data[1], bins, range =  (min_val,max_val))
    
    #normalize histograms
    n_hist0 = hist0.astype(np.float32) / sizes[0]
    n_hist1 = hist1.astype(np.float32) / sizes[1]
    
    return (bin_edges,n_hist0,n_hist1)
    
def survey_column(index, size = 20, bins = 50, force_num = False,\
                  cat = False, ratio = 2, min_p = 0.05,\
                  print_out = False, draw_hist = False):
    name = success_df.columns[index]
    print str(index)+" "+ success_df.columns[index]
    
    s_col = success_df.iloc[:,index]
    f_col = failed_df.iloc[:,index]
    
    #show first size rows
    ccol_df = pd.concat([s_col.iloc[0:size,],f_col.iloc[0:size,]], axis =1)
    #rename columns
    ccol_df.columns =["S:"+name,"F:"+name]
    
    if print_out:
        display(ccol_df)
    
    s_col = s_col.dropna()
    f_col = f_col.dropna()
    
    if (cat):
        unique_vals = np.append(s_col.unique(),f_col.unique())
        unique_vals = np.unique(unique_vals)
        unique_vals = np.sort(unique_vals)
        
        # print out first nr_vals
        nr_vals = 10
        if unique_vals.size >nr_vals:
            print unique_vals[0:nr_vals]
        else:
            print unique_vals
            
        #cast categories into indexes
        s_col = s_col.apply(lambda x: np.searchsorted(unique_vals,x))
        f_col = f_col.apply(lambda x: np.searchsorted(unique_vals,x))

    #force casting into numbers
    elif force_num:
        s_col = s_col.apply(lambda x: pd.to_numeric(x, errors='coerce'))
        f_col = f_col.apply(lambda x: pd.to_numeric(x, errors='coerce'))
  
        s_col = s_col.dropna()
        f_col = f_col.dropna()
    
    #do df summaries
    
    cdes_df = pd.concat([s_col.describe(),f_col.describe()], axis =1)
    #rename columns
    cdes_df.columns =["S:"+name,"F:"+name]
    
    #count how many values there were before preprocessing (before removing weird valeus)
    f_line = pd.DataFrame({"S:"+name: [len(success_df.index)],"F:"+name:[len(failed_df.index)]}, index = ["t_count"])
    
    #prepend fline
    cdes_df = pd.concat([f_line,cdes_df])
    
    if print_out:
        display(cdes_df)

    #get histograms with categorical variables # bins = # categories
    if (cat):
        b, nh_s, nh_f =normalize_double_hist((s_col,f_col),\
                                       sizes = (len(success_df.index),len(failed_df.index)),\
                                       bins = unique_vals.size)
    else:
        b, nh_s, nh_f =normalize_double_hist((s_col,f_col),\
                                       sizes = (len(success_df.index),len(failed_df.index)),\
                                       bins = bins)
    
    if draw_hist:
        fig, ax = plt.subplots(2,2, figsize=(15,6))

        max_hist = max (nh_s.max(), nh_f.max())

        step = b[1] - b[0]

        scale = 1.1

        ax[0,0].bar(b[:-1], nh_s, width= step, color='green')    
        ax[0,0].set_title("S:"+name)
        ax[0,0].set_xlim([b[0]*scale,b[-1]*scale])
        ax[0,0].set_ylim([0,max_hist])

        ax[0,1].bar(b[:-1], nh_f, width= step, color='red')
        ax[0,1].set_title("F:"+name)
        ax[0,1].set_xlim([b[0]*scale,b[-1]*scale])
        ax[0,1].set_ylim([0,max_hist*scale])

        s_less = nh_s *(nh_s <= nh_f)
        ax[1,0].bar(b[:-1], nh_s, width= step, color='green')
        ax[1,0].bar(b[:-1], nh_f, width= step, color='red')
        ax[1,0].bar(b[:-1], s_less, width= step, color='green')
        ax[1,0].set_title("Both")
        ax[1,0].set_xlim([b[0]*scale,b[-1]*scale])
        ax[1,0].set_ylim([0,max_hist*scale])

#     alternative plotting
#     s_diff = nh_s - nh_f
#     ax[1,1].bar(b[:-1], s_diff, width= step, color='orange')
#     ax[1,1].set_title("Diff")
#     ax[1,1].set_xlim([b[0]*scale,b[-1]*scale])
#     ax[1,1].set_ylim([s_diff.min()*scale,s_diff.max()*scale])
    
#     s_more = (nh_s > nh_f)    
#     f_less_val = s_more * nh_f;
#     f_less_val_neg = f_less_val * -1;
    
    
#     ax[1,1].bar(b[:-1], f_less_val_neg, width= step, color='blue')
#     ax[1,1].set_ylim([min(s_diff.min(),f_less_val_neg.min())*scale,s_diff.max()*scale])
    
    
    #calculate ratio between success and fialure
    sf_ratio = nh_s/nh_f
    # remove inf, inf nan
    sf_ratio[np.logical_not(np.isfinite(sf_ratio))] = 0
    #ignore entries below ratio
    sf_ratio[(sf_ratio<ratio)] =  0
    #ignore entries not frequent enough
    sf_ratio[(nh_s <= min_p)] =  0
    
    #only sucessess, no fialures in the bin
    s_pos = np.copy(nh_s)
    
    s_pos[nh_f !=0] = 0

    # to small cut it off
    s_pos[(nh_f ==0) & (nh_s <= min_p)] = 0
    # seems significan keep it
    s_pos[(nh_f ==0) & (nh_s > min_p)] = 1
    
    if draw_hist:    
        if sf_ratio.sum()>0:
            ax[1,1].bar(b[:-1], sf_ratio, width= step, color='blue')
            ax[1,1].set_ylim([0,sf_ratio.max()*scale])
        
        if s_pos.sum()>0:
        #this is scaled to ratio so it is always visible
            ax[1,1].bar(b[:-1], s_pos*ratio, width= step, color='orange')

        ax[1,1].set_title("S/F ratio above "+str(ratio))
        ax[1,1].set_xlim([b[0]*scale,b[-1]*scale])
    
    short_ratio = np.copy(sf_ratio)
    short_ratio = short_ratio[sf_ratio>=ratio]    
    #-1 for sorting desc
    short_ratio = np.sort(-1 *short_ratio)*-1
    
    if short_ratio.size>0:
        if (cat): #(cat and print_out):
            print "vals: " + str(unique_vals[sf_ratio>=ratio])
            print "ratio: " + str(sf_ratio[sf_ratio>=ratio])
            print "suc: " +str(nh_s[sf_ratio>=ratio])
        
        print "sorted ratio: "+str(short_ratio) 
            
        
    
    #sort only the suc > min_p
    only_suc = s_pos * nh_s
    short_suc = np.copy(only_suc)
    short_suc = short_suc[only_suc>0] 
    #-1 for sorting desc
    short_suc = np.sort(-1 *short_suc)*-1
    
    if short_suc.size>0:
        if (cat): #(cat and print_out):
            print "val: " +str(unique_vals[only_suc>0])
            print "no_fail: " +str(nh_s[only_suc>0])
        
        print "sorted no_fail: " +str(short_suc) 
    
 

In [21]:
string_cols = [1, 6, 8]

date_cols=[3, 13, 14]
 
set_cols=[7, 9]

boolean_cols=[2, 12, 24, 27, 29, 30, 31, 32, 34, 35, 38, 46, 47, 48, 49, 50, 51, 52, 53, 54,\
             55, 57, 58, 59, 63, 64, 68, 69, 70, 73, 77, 78, 81, 82, 83, 84, 85, 86, 89, 90,\
             91, 97]

perc_cols = range(102,116)

num_cols=[4, 5, 10, 11, 15, 18, 19, 20, 21, 22, 23, 25, 66, 72, 74, 88, 94, 95, 96, 98, 99,116]

cat_cols=[16, 17, 26, 28, 33, 36, 37, 39, 40, 41, 42, 43, 44, 45, 56, 57, 59, 60, 61, 62, 65,\
          67, 71, 75, 76, 79, 80, 87, 92, 93, 100, 101]


boolean_cols=[2, 12, 24, 27, 29, 30, 31, 32, 34, 35, 38, 46, 47, 48, 49, 50, 51, 52, 53, 54,\
             55, 58, 63, 64, 68, 69, 70, 73, 77, 78, 81, 82, 83, 84, 85, 86, 89, 90,91, 97]


In [234]:
#i_col = 0
#i_col+=1
#i_col=100
#cat_cols[i_col]
print i_col
survey_column(num_cols[i_col],force_num = False, cat = True,\
              ratio = 4, min_p = 0.05,\
              print_out = False, draw_hist = False)
i_col+=1


14
74 Skills score
['0' '10' '100' '11' '12' '13' '13.5' '14' '14.5' '140']
vals: ['14' '18']
ratio: [ 4.92786837  6.02295065]
suc: [ 0.05901639  0.07213115]
sorted ratio: [ 6.02295065  4.92786837]


There are 115 (116? variables) for each company. Shockingly there are almost twice as many (305) sucessful companies as failed ones (167). This is of course *VERY SUSPICIOUS* and likely and indication of a synthetic data set. In real life most companies fail.

This my also indicate that we might address the inverse problem, that is 